In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from collections import Counter

In [3]:
import torch 
import torch.nn as nn
from torchtext import data
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler

In [4]:
# df=pd.read_csv(r'/content/drive/MyDrive/AlgoIntern/Data/discharge_summary_sections.csv')

In [5]:
# df

In [6]:
# df.columns

In [7]:
# complaint_icd=df[['HADM_ID','CATEGORY','DESCRIPTION','chief complaint:','ICD9_CODE','ICD10']]
# medical_history_icd=df[['HADM_ID','CATEGORY','DESCRIPTION','past medical history:','ICD9_CODE','ICD10']]
# allergies_icd=df[['HADM_ID','CATEGORY','DESCRIPTION','allergies:','ICD9_CODE','ICD10']]
# medications_icd=df[['HADM_ID','CATEGORY','DESCRIPTION','discharge medications:','ICD9_CODE','ICD10']]
# social_icd=df[['HADM_ID','CATEGORY','DESCRIPTION','social history:','ICD9_CODE','ICD10']]


In [8]:
# complaint_icd.dropna(subset=['chief complaint:'],inplace=True)
# complaint_icd.reset_index(drop=True,inplace=True)
# medical_history_icd.dropna(subset=['past medical history:'],inplace=True)
# medical_history_icd.reset_index(drop=True,inplace=True)
# allergies_icd.dropna(subset=['allergies:'],inplace=True)
# allergies_icd.reset_index(drop=True,inplace=True)
# medications_icd.dropna(subset=['discharge medications:'],inplace=True)
# medications_icd.reset_index(drop=True,inplace=True)
# social_icd.dropna(subset=['social history:'],inplace=True)
# social_icd.reset_index(drop=True,inplace=True)

In [9]:
# complaint_icd

In [10]:
# def count_description(x):
#   list_description=re.split(',',x)
#   dict_description=dict(Counter(list_description))
#   return dict_description

In [11]:
# complaint_icd_agg=complaint_icd.groupby('HADM_ID',as_index=False).agg({
#     'CATEGORY':'first',
#     'DESCRIPTION':lambda x: ','.join(x),
#     'chief complaint:': lambda x: ', '.join(x),
#     'ICD9_CODE':'first',
#     'ICD10':'first'
#      })
# complaint_icd_agg['description']=complaint_icd_agg['DESCRIPTION'].apply(count_description)
# complaint_icd_agg[['HADM_ID','CATEGORY','description','chief complaint:','ICD9_CODE','ICD10']].to_csv(r'/content/drive/My Drive/AlgoIntern/Data/complaint_icd.csv',index=False)

In [12]:
# medical_history_icd_agg=medical_history_icd.groupby('HADM_ID',as_index=False).agg({
#     'CATEGORY':'first',
#     'DESCRIPTION':lambda x: ','.join(x),
#     'past medical history:': lambda x: ', '.join(x),
#     'ICD9_CODE':'first',
#     'ICD10':'first'
#      })
# medical_history_icd_agg['description']=medical_history_icd_agg['DESCRIPTION'].apply(count_description)
# medical_history_icd_agg[['HADM_ID','CATEGORY','description','past medical history:','ICD9_CODE','ICD10']].to_csv(r'/content/drive/My Drive/AlgoIntern/Data/medical_history_icd.csv',index=False)

In [13]:
# allergies_icd_agg=allergies_icd.groupby('HADM_ID',as_index=False).agg({
#     'CATEGORY':'first',
#     'DESCRIPTION':lambda x: ','.join(x),
#     'allergies:': lambda x: ', '.join(x),
#     'ICD9_CODE':'first',
#     'ICD10':'first'
#      })
# allergies_icd_agg['description']=allergies_icd_agg['DESCRIPTION'].apply(count_description)
# allergies_icd_agg[['HADM_ID','CATEGORY','description','allergies:','ICD9_CODE','ICD10']].to_csv(r'/content/drive/My Drive/AlgoIntern/Data/allergies_icd.csv',index=False)

In [14]:
# medications_icd_agg=medications_icd.groupby('HADM_ID',as_index=False).agg({
#     'CATEGORY':'first',
#     'DESCRIPTION':lambda x: ','.join(x),
#     'discharge medications:': lambda x: ', '.join(x),
#     'ICD9_CODE':'first',
#     'ICD10':'first'
#      })
# medications_icd_agg['description']=medications_icd_agg['DESCRIPTION'].apply(count_description)
# medications_icd_agg[['HADM_ID','CATEGORY','description','discharge medications:','ICD9_CODE','ICD10']].to_csv(r'/content/drive/My Drive/AlgoIntern/Data/medications_icd.csv',index=False)

In [15]:
# social_icd_agg=social_icd.groupby('HADM_ID',as_index=False).agg({
#     'CATEGORY':'first',
#     'DESCRIPTION':lambda x: ','.join(x),
#     'social history:': lambda x: ', '.join(x),
#     'ICD9_CODE':'first',
#     'ICD10':'first'
#      })
# social_icd_agg['description']=social_icd_agg['DESCRIPTION'].apply(count_description)
# social_icd_agg[['HADM_ID','CATEGORY','description','social history:','ICD9_CODE','ICD10']].to_csv(r'/content/drive/My Drive/AlgoIntern/Data/social_icd.csv',index=False)

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [17]:
complaint_icd=pd.read_csv(r'/content/drive/MyDrive/AlgoIntern/Data/complaint_icd.csv')

In [18]:
complaint_icd

,HADM_ID,CATEGORY,description,chief complaint:,ICD9_CODE,ICD10
0,100001.0,Discharge summary,{'Report': 1},right foot bleeding. major surgical or invas...,"25013,3371,5849,5780,V5867,25063,5363,4580,250...","Misc,Misc,N179,Misc,Z794,Misc,Misc,Misc,Misc,I..."
1,100003.0,Discharge summary,{'Report': 1},the patient comes in after a fall. history ...,"53100,2851,07054,5715,5715,5715,45621,53789,40...","Misc,D62,B182,K740,K7460,K7469,Misc,Misc,I10,M..."
2,100006.0,Discharge summary,{'Report': 1},dyspnea. history of present illness: this ...,"49320,51881,51881,486,20300,2761,7850,3090,V12...","Misc,J9600,J9690,J189,Misc,E871,Misc,Misc,Z867..."
3,100007.0,Discharge summary,{'Report': 1},shortness of breath. history of present ill...,"56081,5570,9973,486,4019","Misc,Misc,Misc,J189,I10"
4,100009.0,Discharge summary,{'Report': 1},s/p fall major surgical or invasive procedur...,"41401,99604,4142,25000,27800,V8535,4148,4111,V...","I2510,Misc,Misc,E119,E669,Misc,Misc,I200,Z9861..."
...,...,...,...,...,...,...
40243,199984.0,Discharge summary,{'Report': 1},chest pain - transfer for cath major surgica...,"41071,5990,41401,42731,2449,4019,2724,2724,530...","I214,N390,I2510,I4891,E039,I10,E784,E785,K219,..."
40244,199987.0,Discharge summary,{'Report': 1},s/p motor vehicle crash major surgical or in...,"8080,8604,80709,5180,5180,8082,8056,9190,E8120","Misc,Misc,Misc,J9811,J9819,Misc,Misc,Misc,Misc"
40245,199992.0,Discharge summary,{'Report': 1},hypotension major surgical or invasive proce...,"42789,42789,5070,5849,2760,1120,2449,0539,3272...","I498,R001,J690,N179,E870,Misc,E039,Misc,G4733,..."
40246,199998.0,Discharge summary,{'Report': 1},gist tumor major surgical or invasive proced...,"41401,9971,9971,9971,9971,9975,42731,78820,411...","I2510,I97710,I97790,I9788,I9789,Misc,I4891,Mis..."


In [19]:
complaint_icd['ICD9_CODE_list']=complaint_icd['ICD9_CODE'].apply(lambda x: list(set(re.split(',',x))))
complaint_icd['ICD9_CATEGORY_list']=complaint_icd['ICD9_CODE_list'].apply(lambda x:list(set([re.split("(\d{3}|[VE]\d{2})",i)[1] for i in x]))) # converting codes into high level category
complaint_icd['chief complaint:'].replace(r'\t',' ',regex=True,inplace=True)
complaint_icd['chief complaint:']=complaint_icd['chief complaint:'].apply(lambda x:x.strip())
complaint_icd['chief complaint:']=complaint_icd['chief complaint:'].apply(lambda x:re.sub('\s+',' ',x))

In [20]:
complaint_icd

,HADM_ID,CATEGORY,description,chief complaint:,ICD9_CODE,ICD10,ICD9_CODE_list,ICD9_CATEGORY_list
0,100001.0,Discharge summary,{'Report': 1},right foot bleeding. major surgical or invasiv...,"25013,3371,5849,5780,V5867,25063,5363,4580,250...","Misc,Misc,N179,Misc,Z794,Misc,Misc,Misc,Misc,I...","[40390, 7078, 3371, 25063, 25083, 5849, V1351,...","[250, V13, 458, 362, 707, V58, 578, 403, 536, ..."
1,100003.0,Discharge summary,{'Report': 1},the patient comes in after a fall. history of ...,"53100,2851,07054,5715,5715,5715,45621,53789,40...","Misc,D62,B182,K740,K7460,K7469,Misc,Misc,I10,M...","[4019, 2851, 07054, 53100, 7823, 53789, 45621,...","[456, 782, 537, 531, 571, 535, 285, 070, 401]"
2,100006.0,Discharge summary,{'Report': 1},dyspnea. history of present illness: this is a...,"49320,51881,51881,486,20300,2761,7850,3090,V12...","Misc,J9600,J9690,J189,Misc,E871,Misc,Misc,Z867...","[49320, V1251, 7850, V1582, 486, 3090, 51881, ...","[518, 276, 785, 493, 486, V12, 309, 203, V15]"
3,100007.0,Discharge summary,{'Report': 1},shortness of breath. history of present illnes...,"56081,5570,9973,486,4019","Misc,Misc,Misc,J189,I10","[9973, 56081, 486, 5570, 4019]","[997, 486, 557, 560, 401]"
4,100009.0,Discharge summary,{'Report': 1},s/p fall major surgical or invasive procedure:...,"41401,99604,4142,25000,27800,V8535,4148,4111,V...","I2510,Misc,Misc,E119,E669,Misc,Misc,I200,Z9861...","[4019, 4111, V8535, V1582, 4142, 41401, 44021,...","[996, 250, V45, 426, 440, V58, 411, V85, 285, ..."
...,...,...,...,...,...,...,...,...
40243,199984.0,Discharge summary,{'Report': 1},chest pain - transfer for cath major surgical ...,"41071,5990,41401,42731,2449,4019,2724,2724,530...","I214,N390,I2510,I4891,E039,I10,E784,E785,K219,...","[3659, 42731, 2449, 2724, 49390, V1588, 41401,...","[493, 401, 244, 715, 599, 427, 365, 272, 530, ..."
40244,199987.0,Discharge summary,{'Report': 1},s/p motor vehicle crash major surgical or inva...,"8080,8604,80709,5180,5180,8082,8056,9190,E8120","Misc,Misc,Misc,J9811,J9819,Misc,Misc,Misc,Misc","[5180, 8604, E8120, 9190, 80709, 8082, 8080, 8...","[518, 807, 808, 805, 919, E81, 860]"
40245,199992.0,Discharge summary,{'Report': 1},hypotension major surgical or invasive procedu...,"42789,42789,5070,5849,2760,1120,2449,0539,3272...","I498,R001,J690,N179,E870,Misc,E039,Misc,G4733,...","[2449, 32723, E9352, 5849, E9363, 78720, 2859,...","[787, 327, 276, 443, 244, 564, 053, E93, 285, ..."
40246,199998.0,Discharge summary,{'Report': 1},gist tumor major surgical or invasive procedur...,"41401,9971,9971,9971,9971,9975,42731,78820,411...","I2510,I97710,I97790,I9788,I9789,Misc,I4891,Mis...","[78820, 42731, 4019, 4111, V4589, 2724, 41401,...","[E87, 518, V45, 997, 443, 440, 788, 411, 427, ..."


In [21]:
top10_icd9category=['401', '427', '276', '272', '250', '414', '428', '518', '285', '584']

In [ ]:
vocabulary={'!': 2, '"': 180, '#': 4437, '$': 1, '%': 1112, '&': 437, "'": 1822, '(': 5757, ')': 8852, '*': 1355, '+': 423, ',': 37548, '-': 25719, '.': 74329, '/': 27690, '0': 7423, '1': 19826, '2': 16458, '3': 13831, '4': 9255, '5': 8362, '6': 6901, '7': 4156, '8': 3214, '9': 2750, ':': 35177, ';': 1029, '<': 27891, '=': 779, '>': 28328, '?': 1087, '@': 47, '[': 258, '\\': 10, ']': 261, '^': 209, '_': 395, '`': 2, 'a': 484587, 'b': 78357, 'c': 229891, 'd': 188068, 'e': 532471, 'f': 82452, 'g': 75634, 'h': 150403, 'i': 454344, 'j': 5220, 'k': 17925, 'l': 231597, 'm': 158976, 'n': 327420, 'o': 351372, 'p': 185722, 'q': 5435, 'r': 407161, 's': 350968, 't': 388087, 'u': 142782, 'v': 45552, 'w': 30437, 'x': 15415, 'y': 145744, 'z': 4150, '~': 81}

In [ ]:
def labeltarget(x):
  target=np.zeros(10,dtype="float32")
  for index,code in enumerate(top10_icd9category):
    if code in x :
      target[index]=1
  return target

In [ ]:
def character_embedding(word,input_size=500):
  embedding_matrix=np.zeros((65,input_size),dtype="float32")
  for i in range(len(word)):
    if i>input_size-1:
      break
    if word[i]!=' ':
      index_one_hot=list(vocabulary).index(word[i])
      embedding_matrix[index_one_hot][i]=1
    
  return embedding_matrix

In [ ]:
import nltk
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')).union(set(string.punctuation)).union(set(list(range(10)))) # stopwords + punctuation+ numbers

In [ ]:
def preprocessing(text):
  words=word_tokenize(text)
  filtered_sentence = [] 
  # remove stopwords
  for word in words: 
    if word not in stop_words: 
        filtered_sentence.append(word) 
  
  # lemmatize
  lemma_word = []
  wordnet_lemmatizer = WordNetLemmatizer()
  for w in filtered_sentence:
    word1 = wordnet_lemmatizer.lemmatize(w, pos = "n")
    word2 = wordnet_lemmatizer.lemmatize(word1, pos = "v")
    word3 = wordnet_lemmatizer.lemmatize(word2, pos = ("a"))
    lemma_word.append(word3)
  return lemma_word

